In [10]:
import re
import linecache
from time import process_time
from collections import deque
import array as arr
from pathlib import Path


class MultiInsieme:

    def __init__(self, cardinalita):
        if cardinalita == 0:
            self.lista_array = []
        else:
            self.lista_array = [arr.array('H', []) for _ in range(cardinalita)]
        self.cardinalita = cardinalita

    def __len__(self):
        if not self.lista_array:
            return 0
        return len(self.lista_array[0])

    def get(self, indice):
        if not self.lista_array:
            return arr.array('H', [])
        return arr.array('H', [self.lista_array[i][indice] for i in range(self.cardinalita)])

    def aggiungi_insiemi(self, base, elementi):
        if not len(base):
            self.lista_array[0].extend(elementi)
        else:
            for i in range(len(base)):
                self.lista_array[i].extend(arr.array('H', [base[i]] * len(elementi)))
            self.lista_array[len(base)].extend(elementi)


#*********************************************************************#

#Legge il file, estrae la matrice e la ritorna (lista di array)
def leggi_matrice(percorso_file):
    matrice = []
    with percorso_file.open('r') as file:
        for riga in file:
            if riga[0] != ";":
                nuova_riga = riga.replace("-", "")
                elementi = nuova_riga.split(" ")
                elementi.remove("\n")
                matrice.append(arr.array('B', list(map(int, elementi))))
    return matrice


def leggi_dominio(nome_file):
    riga_dominio = linecache.getline(nome_file, 5)
    dominio = re.findall('\(([^)]+)', riga_dominio)
    return dominio, riga_dominio


def carica_matrice(percorso_file):
    print(f'Caricamento matrice dal file {str(percorso_file)}')
    array = leggi_matrice(percorso_file)
    righe = len(array)
    colonne = len(array[0])
    nome_matrice = percorso_file.stem
    dominio_matrice, linea_dominio = leggi_dominio(str(percorso_file))
    if len(array[0]) == len(dominio_matrice):
        print('Matrice caricata con successo')
        print(f'Matrice: {nome_matrice}')
        print(f'Numero righe: {righe}')
        print(f'Numero colonne: {colonne}')
        dominio_base = arr.array('H', list(range(0, len(array[0]))))
        return nome_matrice, array, dominio_base, linea_dominio, True
    else:
        print('Errore sulle dimensioni del file')
        return nome_matrice, [], [], linea_dominio, False


#************************* ALG BASE ************************#

def succ(elem):
    if elem == -1:
        return 1
    else:
        return elem + 1


def max_insieme(insieme):
    if not insieme:
        return -1
    else:
        return insieme[-1]


def alg_base(matrice, dominio):
    start_time = process_time()
    n_iter = 0
    lista_mhs = []
    coda = deque()
    valore_max = len(matrice[0])
    coda.append(MultiInsieme(0))
    try:
        while coda:
            multi_insieme = coda.popleft()
            multi_insieme_ok = MultiInsieme(multi_insieme.cardinalita + 1)
            multi_insieme_mhs = MultiInsieme(multi_insieme.cardinalita + 1)
            len_multi_insieme = len(multi_insieme)
            if not len_multi_insieme:  # Insieme vuoto, devo ottenere i singoletti
                len_multi_insieme += 1
            for i in range(len_multi_insieme):
                insieme = multi_insieme.get(i)

                m = max_insieme(insieme)
                e = succ(m)

                ok_array = arr.array('H', [])
                mhs_array = arr.array('H', [])
                for elem in range(e, valore_max + 1):
                    n_iter += 1
                    vett_rapp = crea_vett_rapp(insieme, elem, matrice)
                    result = check(insieme, elem, vett_rapp)
                    if result == "ok" and elem != valore_max:
                        ok_array.append(elem)
                    elif result == "mhs":
                        mhs_array.append(elem)
                if ok_array:
                    multi_insieme_ok.aggiungi_insiemi(insieme, ok_array)
                if mhs_array:
                    multi_insieme_mhs.aggiungi_insiemi(insieme, mhs_array)
            if len(multi_insieme_ok):
                coda.append(multi_insieme_ok)
            if len(multi_insieme_mhs):
                lista_mhs.append(multi_insieme_mhs)
    except KeyboardInterrupt:
        print("Esecuzione terminata dall'utente\n")
        output(lista_mhs, dominio)
        return lista_mhs, process_time() - start_time, n_iter                 
    output(lista_mhs, dominio)
    return lista_mhs, process_time() - start_time, n_iter


def check(insieme, elem, vett_rapp):
    proiezione = set(vett_rapp)

    if elem not in proiezione:
        return "ko"

    for i in insieme:
        if i not in proiezione:
            return "ko"

    if 0 in proiezione:
        return "ok"
    else:
        return "mhs"


def crea_vett_rapp(insieme, elem, matrice):
    vett_rapp = arr.array('H', [matrice[i][elem - 1] * elem for i in range(len(matrice))])
    for i in range(len(matrice)):
        for j in insieme:
            if matrice[i][j - 1]:
                if vett_rapp[i]:
                    vett_rapp[i] = 65535
                    break
                else:
                    vett_rapp[i] = j
    return vett_rapp


def output(lista_mhs, dominio):
    for multi_mhs in lista_mhs:
        for riga in multi_mhs.lista_array:
            for i in range(len(riga)):
                riga[i] = dominio[riga[i] - 1]       
           

#********************************* PRE ELAB *************************************#


def contiene(matrice, i, j):
    for k in range(len(matrice[0])):
        if (not matrice[i][k]) and matrice[j][k]:
            return False
    return True


def costruisci_array(matrice):
    lista = []
    for i in range(len(matrice)):  # Numero di righe
        lista.append((sum(matrice[i]), i))
    lista.sort(key=lambda x: x[0], reverse=True)
    return arr.array('H', list(zip(*lista))[1])


def togli_righe(matrice):
    indici_rimossi = []
    array = costruisci_array(matrice)
    j = len(array)
    for i in range(j):
        for k in range(i + 1, j):
            if contiene(matrice, array[i], array[k]):
                indici_rimossi.append(array[i])
                break

    indici_rimossi.sort(reverse=True)
    for i in indici_rimossi:
        del matrice[i]
    indici_rimossi = arr.array('H', indici_rimossi)
    return indici_rimossi, len(indici_rimossi)


def colonna_di_zero(matrice, i):
    for j in range(len(matrice)):
        if matrice[j][i]:
            return False
    return True


def togli_colonne(matrice):
    indici_rimossi = []
    for i in range(len(matrice[0]) - 1, -1, -1):
        if colonna_di_zero(matrice, i):
            indici_rimossi.append(i)
    for i in indici_rimossi:
        deque(map(lambda x: x.pop(i), matrice), maxlen=0)
    indici_rimossi = arr.array('H', indici_rimossi)
    return indici_rimossi, len(indici_rimossi)


def pre_elaborazione_dominio(dominio, indici_da_rimuovere):
    if indici_da_rimuovere:
        for i in indici_da_rimuovere:
            del dominio[i]


def alg_con_pre(matrice, dominio):
    righe_rimosse, numero_righe_rimosse = togli_righe(matrice)
    colonne_rimosse, numero_colonne_rimosse = togli_colonne(matrice)
    pre_elaborazione_dominio(dominio, colonne_rimosse)
    print("Numero righe rimosse: ", numero_righe_rimosse) 
    print("Numero colonne rimosse: ", numero_colonne_rimosse)
    print(f"Gli indici di riga rimossi sono: {stringa_da_array(righe_rimosse, a_capo=False)}")
    print(f"Gli indici di colonna rimossi sono: {stringa_da_array(sorted(colonne_rimosse), a_capo=False)}")
    lista_mhs, tempo_di_esecuzione, n_iter = alg_base(matrice, dominio)
    return lista_mhs, tempo_di_esecuzione, n_iter

#**************************************************************************#

def max_min_mhs(lista_mhs):
    if lista_mhs:
        def dimensione(x):
            return x.cardinalita
        return (max(map(dimensione, lista_mhs))), (min(map(dimensione, lista_mhs)))
    else:
        return 0, 0


def numero_mhs(lista_mhs):
    totale = 0
    for multi_mhs in lista_mhs:
        totale += len(multi_mhs)
    return totale


#**********************************************************************#


def stringa_da_array(array, a_capo=True):
    if array:
        separatore = " "
        stringa = separatore.join([str(elem) for elem in array])
        if a_capo:
            return f'{stringa} -\n'
        else:
            return stringa
    else:
        return '\n'


def stampa_riepilogo(tempo, n_iterazioni, totale_mhs, min_mhs, max_mhs):
    print(f"Il tempo richiesto dall'esecuzione e' stato di {tempo} s")
    print(f"Il numero di iterazioni compiute e' stato di {n_iterazioni}")
    #print(f"La massima occupazione di memoria e' stata di {max_memoria} MiB")
    print(f"Sono stati trovati {totale_mhs} MHS")
    print(f"La cardinalita' minima dei MHS trovati e' {min_mhs}")
    print(f"La cardinalita' massima dei MHS trovati e' {max_mhs}\n")


def esegui_alg_pre(percorso_file):
    _, matrice, dominio, _, esito= carica_matrice(percorso_file)
    if not esito: 
        return 0
    else: 
        lista_mhs, tempo_di_esecuzione, n_iter = alg_con_pre(matrice, dominio)
        max_mhs ,min_mhs = max_min_mhs(lista_mhs)
        stampa_riepilogo(tempo_di_esecuzione, n_iter, numero_mhs(lista_mhs), min_mhs, max_mhs)   


def esegui_alg_base(percorso_file):
    _, matrice, dominio, _, esito = carica_matrice(percorso_file)
    if not esito: 
        return 0
    else: 
        lista_mhs, tempo_di_esecuzione, n_iter=alg_base(matrice, dominio)
        max_mhs, min_mhs = max_min_mhs(lista_mhs)
        stampa_riepilogo(tempo_di_esecuzione, n_iter, numero_mhs(lista_mhs), min_mhs, max_mhs)

# **Esecuzione, caricare il file e incollare il suo percorso in "path"**





In [11]:
#ESECUZIONE ALG. BASE
percorso_file = Path("/content/74283.010.matrix")

soluzioni = esegui_alg_base(percorso_file)

Caricamento matrice dal file /content/74283.010.matrix
Matrice caricata con successo
Matrice: 74283.010
Numero righe: 9
Numero colonne: 36
Il tempo richiesto dall'esecuzione e' stato di 0.10668800599999972 s
Il numero di iterazioni compiute e' stato di 11312
Sono stati trovati 247 MHS
La cardinalita' minima dei MHS trovati e' 2
La cardinalita' massima dei MHS trovati e' 4



In [12]:
# ESECUZIONE CON PRE-ELABORAZIONE
percorso_file = Path("/content/74181.014.matrix")

sol = esegui_alg_pre(percorso_file)

Caricamento matrice dal file /content/74181.014.matrix
Matrice caricata con successo
Matrice: 74181.014
Numero righe: 3
Numero colonne: 65
Numero righe rimosse:  0
Numero colonne rimosse:  44
Gli indici di riga rimossi sono: 

Gli indici di colonna rimossi sono: 1 3 4 5 6 7 8 9 13 14 15 16 17 18 19 21 22 23 25 26 29 30 33 34 35 36 37 38 39 40 41 42 43 48 49 50 53 54 55 56 58 59 60 62
Il tempo richiesto dall'esecuzione e' stato di 0.0019931520000007197 s
Il numero di iterazioni compiute e' stato di 291
Sono stati trovati 19 MHS
La cardinalita' minima dei MHS trovati e' 2
La cardinalita' massima dei MHS trovati e' 3

